# Installing MASE (again)

Run the block below to install MASE in the current Colab runtime

In [ ]:
git_token = "YOUR_GIT_TOKEN"
short_code = "YOUR_SHORT_CODE"

# Check the current python version (It should be using Python 3.10) and update pip to the latest version.
!python --version
!python -m pip install --user --upgrade pip

# Clone MASE from your branch (the branch must already exist)
!git clone -b lab1_{short_code} https://{git_token}@github.com/DeepWok/mase.git

# Install requirements
!python -m pip install -r ./mase/machop/requirements.txt

# Change working directory to machop
%cd ./mase/machop/

In [ ]:
!./ch --help

# General introduction

In this lab, you will learn how to use the software stack of MASE. There are in total 7 tasks you would need to finish, and 1 optional task.

# Turning you network to a graph

One specific feature of MASE is its capability to transform DL models to a computation graph using the [torch.fx](<https://pytorch.org/docs/stable/fx.html>) framework.


## Use the Transform functionality without CLI

This tutorial describes how to use the MASE transform functionality for a pre-trained model.

## Import related packages and machop

In [26]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

In [27]:
# Change working directory to machop
%cd ./machop/

[Errno 2] No such file or directory: './machop/'
/home/taa123/mase/machop


In [28]:
from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph import (
    save_node_meta_param_interface_pass,
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph

from chop.models import get_model_info, get_model

set_logging_verbosity("info")

INFO     Set logging level to info


## Set up the dataset

Here we create a `MaseDataModule` using the `jsc` dataset from lab1. Note the `MaseDataModule` also requires the name of the model you plan to use data module with. In this case it is `jsc-tiny`.

In [29]:
batch_size = 8
model_name = "jsc-tiny"
dataset_name = "jsc"


data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()


## Set up the model 

Here we use the previously trained `jsc-tiny` model in lab 1 as an example.

In [5]:
# If you stored your model checkpoint on Google Drive, remember to mount the drive to the current runtime in order to access it
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [30]:
# 📝️ change this CHECKPOINT_PATH to the one you trained in Lab1
CHECKPOINT_PATH = "../mase_output/jsc-tiny_classification_jsc_2024-02-04/software/training_ckpts/best.ckpt"
model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

model = load_model(load_name=CHECKPOINT_PATH, load_type="pl", model=model)

INFO     Loaded pytorch lightning checkpoint from ../mase_output/jsc-tiny_classification_jsc_2024-02-04/software/training_ckpts/best.ckpt


# Get a dummy data in
With the dataset module and model information, we can grab an input generator.

In [31]:
# get the input generator
input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# a demonstration of how to feed an input value to the model
dummy_in = next(iter(input_generator))
_ = model(**dummy_in)


## Generate a MaseGraph
We have two forms of passes: transform passes and analysis passes, both of them would require the model to be transferred into a MaseGraph to allow manipulation.

In [32]:
# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

## Running an Analysis pass
Analysis pass DOES NOT change the graph

The following analysis passes are essential to prepare the graph for other passes

In [33]:
mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

We will first run a simple graph analysis to understand the structure of the model.

In [34]:
# report graph is an analysis pass that shows you the detailed information in the graph
from chop.passes.graph import report_graph_analysis_pass
_ = report_graph_analysis_pass(mg)

graph():
    %x : [num_users=1] = placeholder[target=x]
    %seq_blocks_0 : [num_users=1] = call_module[target=seq_blocks.0](args = (%x,), kwargs = {})
    %seq_blocks_1 : [num_users=1] = call_module[target=seq_blocks.1](args = (%seq_blocks_0,), kwargs = {})
    %seq_blocks_2 : [num_users=1] = call_module[target=seq_blocks.2](args = (%seq_blocks_1,), kwargs = {})
    %seq_blocks_3 : [num_users=1] = call_module[target=seq_blocks.3](args = (%seq_blocks_2,), kwargs = {})
    return seq_blocks_3
Network overview:
{'placeholder': 1, 'get_attr': 0, 'call_function': 0, 'call_method': 0, 'call_module': 4, 'output': 1}
Layer types:
[BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace=True), Linear(in_features=16, out_features=5, bias=True), ReLU(inplace=True)]


## Running another Analysis pass: Profile statistics

The pass `profile_statistics_analysis_pass` collects statistics of parameters and activations, and save them to node's metadata.

Here is a list of all the supported statistics. Refer to the `__init__` of statistic classes in `chop.passes.analysis.statistical_profiler.stat` to check the args each stat class takes.

This is a more complex analysis than the previous pass, and thus it would require you to pass in additional arguments for this pass.

### Example: the range of weights & input activations of nodes

Say we want to collect the tensor-wise min-max range of the 1st `torch.nn.Linear` nodes' weights & bias, and the channel-wise 97% quantile min-max of the 1st `torch.nn.Linear` nodes' input activations. We can do the following:

In [35]:

pass_args = {
    "by": "type",                                                            # collect statistics by node name
    "target_weight_nodes": ["linear"],                                       # collect weight statistics for linear layers
    "target_activation_nodes": ["relu"],                                     # collect activation statistics for relu layers
    "weight_statistics": {
        "variance_precise": {"device": "cpu", "dims": "all"},                # collect precise variance of the weight
    },
    "activation_statistics": {
        "range_quantile": {"device": "cpu", "dims": "all", "quantile": 0.97} # collect 97% quantile of the activation range
    },
    "input_generator": input_generator,                                      # the input generator for feeding data to the model
    "num_samples": 32,                                                       # feed 32 samples to the model
}

We can use the `report_node_meta_param_analysis_pass` to inspect the collected statistics.

In [36]:
mg, _ = profile_statistics_analysis_pass(mg, pass_args)
mg, _ = report_node_meta_param_analysis_pass(mg, {"which": ("software",)})

Profiling act statistics: 100%|██████████| 4/4 [00:00<00:00, 365.91it/s]
INFO     Inspecting graph [add_common_meta_param_analysis_pass]
INFO     
+--------------+--------------+---------------------+--------------+-----------------------------------------------------------------------------------------+
| Node name    | Fx Node op   | Mase type           | Mase op      | Software Param                                                                          |
+==============+==============+=====================+==============+=========================================================================================+
| x            | placeholder  | placeholder         | placeholder  | {'results': {'data_out_0': {'stat': {}}}}                                               |
+--------------+--------------+---------------------+--------------+-----------------------------------------------------------------------------------------+
| seq_blocks_0 | call_module  | module              | batc

## Running a Transform pass: Quantisation

As its name suggests, the transform pass would modify the `MaseGraph`.
Similar to the previous analysis pass example, we would need to first declare the configuration for the pass.

In [37]:
pass_args = {
    "by": "type",
    "default": {"config": {"name": None}},
    "linear": {
        "config": {
            "name": "integer",
            # data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # weight
            "weight_width": 8,
            "weight_frac_width": 4,
            # bias
            "bias_width": 8,
            "bias_frac_width": 4,
        }
    },
}

We can then proceed to apply the transformation, in this case, we kept the original graph on purpose, so that we can print a `diff`.

In [38]:
from chop.passes.graph.transforms import (
    quantize_transform_pass,
    summarize_quantization_analysis_pass,
)
from chop.ir.graph.mase_graph import MaseGraph


ori_mg = MaseGraph(model=model)
ori_mg, _ = init_metadata_analysis_pass(ori_mg, None)
ori_mg, _ = add_common_metadata_analysis_pass(ori_mg, {"dummy_in": dummy_in})

mg, _ = quantize_transform_pass(mg, pass_args)
summarize_quantization_analysis_pass(ori_mg, mg, save_dir="quantize_summary")

INFO     Quantized graph histogram:
INFO     
| Original type   | OP           |   Total |   Changed |   Unchanged |
|-----------------+--------------+---------+-----------+-------------|
| BatchNorm1d     | batch_norm1d |       1 |         0 |           1 |
| Linear          | linear       |       1 |         1 |           0 |
| ReLU            | relu         |       2 |         0 |           2 |
| output          | output       |       1 |         0 |           1 |
| x               | placeholder  |       1 |         0 |           1 |


In [39]:
from chop.passes.graph.utils import get_mase_op, get_mase_type, get_node_actual_target

def get_type_str(node):
        if node.op == "call_module":
            return type(get_node_actual_target(node)).__name__
        elif get_mase_type(node) in [
            "builtin_func",
            "module_related_func",
            "patched_func",
        ]:
            return get_node_actual_target(node).__name__
        elif get_mase_type(node) in ["implicit_func"]:
            actual_target = get_node_actual_target(node)
            if isinstance(actual_target, str):
                return actual_target
            else:
                return actual_target.__name__
        else:
            return node.target

row1 = []
row2 = []

for ori_n, n in zip(ori_mg.fx_graph.nodes, mg.fx_graph.nodes):

    if(get_type_str(ori_n) != get_type_str(n)):

        row1.append(
            [
                get_mase_op(ori_n),
                get_type_str(ori_n),
            ]
        )
        row2.append(
            [
                get_mase_op(n),
                get_type_str(n),
            ]
        )

print(row1)
print(row2)


[['linear', 'Linear']]
[['linear', 'LinearInteger']]


### Verifying the quantization

In [47]:
for node, ori_node in zip(mg.fx_graph.nodes, ori_mg.fx_graph.nodes) :
    if get_mase_op(ori_node) == "linear":
        print("\noriginal nodes :\n")
        print("precision : " + str(ori_node.meta["mase"].parameters["common"]["args"]["weight"]["precision"]))
        print("weights : " + str(ori_node.meta["mase"].parameters["common"]["args"]["weight"]["value"]))
        print("bias : " + str(ori_node.meta["mase"].parameters["common"]["args"]["bias"]["value"]))
        
        print("quantized nodes :\n")
        print("precision : " + str(node.meta["mase"].parameters["common"]["args"]["weight"]["precision"]))
        print("weights and bias : ")
        tnode = get_node_actual_target(node)
        tnode.quantize_weights()


original nodes :

precision : [32]
weights : Parameter containing:
tensor([[-0.2631,  0.2186, -0.1505,  0.0112, -0.0510,  0.1241,  0.0753,  0.3352,
          0.0506,  0.3139,  0.1904,  0.1726,  0.0161,  0.0136,  0.0656,  0.4987],
        [ 0.5028, -0.1142, -0.6989, -0.3384, -0.1510,  0.0972,  0.0224,  0.2828,
          0.0307,  0.1161,  0.3328, -0.0473, -0.1052,  0.0879, -0.7038, -0.2234],
        [ 0.0641,  0.2645, -0.1384, -0.7932, -0.1599,  0.0362,  0.1388,  0.0398,
          0.0338, -0.1022,  0.1453, -0.1785, -0.1806, -0.1287,  0.3045, -0.3323],
        [ 0.0312,  0.0241,  0.0315,  0.0811,  0.1490, -0.0377,  0.3045, -0.0989,
          0.1824, -0.1918, -0.1906, -0.1665,  0.1237,  0.1032, -0.0797, -0.1074],
        [-0.2281, -0.0245,  0.2733,  0.3295,  0.3644,  0.4934, -0.2790, -0.0075,
          0.0163,  0.1082,  0.0182,  0.1842, -0.0623, -0.1322,  0.1295, -0.1515]],
       requires_grad=True)
bias : Parameter containing:
tensor([-0.1564,  0.2497,  0.6091, -0.1329, -0.1206], requir



# Exercises:

We have now seen how to:
1. Set up a dataset
2. Set up a model
3. Generate a `MaseGraph` from the model
4. Run Analysis and Transform passes on the `MaseGraph`

Now consider the following problems:

1. Explain the functionality of `report_graph_analysis_pass` and its printed jargons such as `placeholder`, `get_attr` ... You might find the doc of [torch.fx](https://pytorch.org/docs/stable/fx.html) useful.

2. What are the functionalities of `profile_statistics_analysis_pass` and `report_node_meta_param_analysis_pass` respectively?

## MASE OPs and MASE Types

MASE is designed to be a very high-level intermediate representation (IR), this is very different from the classic [LLVM IR](https://llvm.org/docs/LangRef.html) that you might be familiar with.

The following MASE Types are available:
(Note from Aaron: do we have a page somewhere that have summarized this?)


## A deeper dive into the quantisation transform

3. Explain why only 1 OP is changed after the `quantize_transform_pass` .

4. Write some code to traverse both `mg` and `ori_mg`, check and comment on the nodes in these two graphs. You might find the source code for the implementation of `summarize_quantization_analysis_pass` useful.

5. Perform the same quantisation flow to the bigger JSC network that you have trained in lab1. You must be aware that now the `pass_args` for your custom network might be different if you have used more than the `Linear` layer in your network.

6. Write code to show and verify that the weights of these layers are indeed quantised. You might need to go through the source code of the implementation of the quantisation pass and also the implementation of the [Quantized Layers](../../machop/chop/passes/transforms/quantize/quantized_modules/linear.py) .

## The command line interface

The same flow can also be executed on the command line throw the `transform` action.

```bash
# make sure you have the same printout
pwd
# it should show
# your_dir/mase-tools/machop

# enter the following command
./ch transform --config configs/examples/jsc_toy_by_type.toml --task cls --cpu=0
```
7. Load your own pre-trained JSC network, and perform perform the quantisation using the command line interface.

## \[Optional] Write your own pass

Many examples of existing passes are in the [source code](../..//machop/chop/passes/__init__.py), the [test files](../../machop/test/passes) for these passes also contain useful information on helping you to understand how these passes are used.

Implement a pass to count the number of FLOPs (floating-point operations) and BitOPs (bit-wise operations).

### Loading my network

In [48]:
batch_size = 8
model_name = "jsc-thomas"
dataset_name = "jsc"


data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()

CHECKPOINT_PATH = "../mase_output/thomas/software/training_ckpts/best.ckpt"
model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

model = load_model(load_name=CHECKPOINT_PATH, load_type="pl", model=model)

# get the input generator
input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# a demonstration of how to feed an input value to the model
dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

INFO     Loaded pytorch lightning checkpoint from ../mase_output/thomas/software/training_ckpts/best.ckpt


### Applying the quantization

In [10]:
pass_args = {
    "by": "type",
    "default": {"config": {"name": None}},
    "linear": {
        "config": {
            "name": "integer",
            # data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # weight
            "weight_width": 8,
            "weight_frac_width": 4,
            # bias
            "bias_width": 8,
            "bias_frac_width": 4,
        }
    },
}

from chop.passes.graph.transforms import (
    quantize_transform_pass,
    summarize_quantization_analysis_pass,
)
from chop.ir.graph.mase_graph import MaseGraph


ori_mg = MaseGraph(model=model)
ori_mg, _ = init_metadata_analysis_pass(ori_mg, None)
ori_mg, _ = add_common_metadata_analysis_pass(ori_mg, {"dummy_in": dummy_in})

mg, _ = quantize_transform_pass(mg, pass_args)
summarize_quantization_analysis_pass(ori_mg, mg, save_dir="quantize_summary")

INFO     Quantized graph histogram:
INFO     
| Original type   | OP           |   Total |   Changed |   Unchanged |
|-----------------+--------------+---------+-----------+-------------|
| BatchNorm1d     | batch_norm1d |       4 |         0 |           4 |
| Linear          | linear       |       3 |         3 |           0 |
| ReLU            | relu         |       4 |         0 |           4 |
| output          | output       |       1 |         0 |           1 |
| x               | placeholder  |       1 |         0 |           1 |


### Verifying the difference

In [11]:
from chop.passes.graph.utils import get_mase_op, get_mase_type, get_node_actual_target

def get_type_str(node):
        if node.op == "call_module":
            return type(get_node_actual_target(node)).__name__
        elif get_mase_type(node) in [
            "builtin_func",
            "module_related_func",
            "patched_func",
        ]:
            return get_node_actual_target(node).__name__
        elif get_mase_type(node) in ["implicit_func"]:
            actual_target = get_node_actual_target(node)
            if isinstance(actual_target, str):
                return actual_target
            else:
                return actual_target.__name__
        else:
            return node.target

row1 = []
row2 = []

for ori_n, n in zip(ori_mg.fx_graph.nodes, mg.fx_graph.nodes):

    if(get_type_str(ori_n) != get_type_str(n)):

        row1.append(
            [
                get_mase_op(ori_n),
                get_type_str(ori_n),
            ]
        )
        row2.append(
            [
                get_mase_op(n),
                get_type_str(n),
            ]
        )

print(row1)
print(row2)


[['linear', 'Linear'], ['linear', 'Linear'], ['linear', 'Linear']]
[['linear', 'LinearInteger'], ['linear', 'LinearInteger'], ['linear', 'LinearInteger']]


### Verifying the quantization 

In [15]:
for node, ori_node in zip(mg.fx_graph.nodes, ori_mg.fx_graph.nodes) :
    if get_mase_op(ori_node) == "linear":
        print("\noriginal nodes :\n")
        print("precision : " + str(ori_node.meta["mase"].parameters["common"]["args"]["weight"]["precision"]))
        print("weights : " + str(ori_node.meta["mase"].parameters["common"]["args"]["weight"]["value"]))
        print("bias : " + str(ori_node.meta["mase"].parameters["common"]["args"]["bias"]["value"]))
        
        print("quantized nodes :\n")
        print("precision : " + str(node.meta["mase"].parameters["common"]["args"]["weight"]["precision"]))
        print("weights and bias : ")
        tnode = get_node_actual_target(node)
        tnode.quantize_weights()


original nodes :

{'shape': [8, 16], 'torch_dtype': torch.float32, 'type': 'integer', 'precision': [8, 4], 'value': tensor([[1.0821, 0.5521, 1.8773, 1.5970, 0.5708, 0.2627, 0.0000, 0.2967, 0.0000,
         0.0000, 0.5118, 0.1721, 0.0000, 0.1285, 2.1937, 0.3275],
        [0.6808, 0.4891, 0.5599, 0.0000, 0.5556, 0.0238, 0.2055, 0.9447, 0.4924,
         0.0711, 1.2118, 0.7547, 0.3876, 0.4340, 0.0000, 1.7036],
        [0.0187, 1.0708, 0.7984, 0.0000, 1.0500, 0.7307, 0.1370, 1.2431, 0.4306,
         0.4630, 1.7967, 1.3528, 0.6853, 0.5884, 0.0000, 1.9584],
        [0.9307, 0.6949, 1.9051, 2.0185, 1.5252, 0.8979, 0.0000, 0.3463, 0.0000,
         0.0000, 0.6979, 0.2402, 0.0000, 0.1628, 2.1317, 0.5314],
        [1.3897, 0.0000, 0.3248, 0.0000, 0.0000, 0.0000, 0.0000, 0.5998, 0.0587,
         0.0000, 0.9564, 0.7599, 0.0000, 0.3161, 0.0000, 0.6843],
        [1.1016, 0.0000, 0.1690, 0.0000, 0.0000, 0.0000, 0.0000, 0.9263, 0.2312,
         0.0000, 0.9434, 1.0882, 0.0000, 0.4613, 0.0000, 1.1430],
 

In [52]:
for node, ori_node in zip(mg.fx_graph.nodes, ori_mg.fx_graph.nodes) :
    if get_mase_op(ori_node) == "linear":
        print(str(ori_node.meta["mase"].parameters))

{'common': {'mase_type': 'module_related_func', 'mase_op': 'linear', 'args': {'data_in_0': {'shape': [8, 16], 'torch_dtype': torch.float32, 'type': 'float', 'precision': [32], 'value': tensor([[0.5620, 0.0000, 1.1657, 0.9408, 0.6311, 0.9011, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.2554, 0.2444],
        [0.3116, 0.5497, 1.6658, 1.6405, 0.7305, 0.5714, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.8363, 0.0000],
        [1.6901, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0987, 0.0000,
         0.5503, 0.7937, 0.8760, 0.0480, 1.0274, 0.0000, 0.0000],
        [1.8647, 0.0000, 0.0000, 0.0000, 0.0963, 0.0000, 2.3340, 2.1436, 2.1876,
         0.3290, 0.0000, 0.0000, 0.1350, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.6008, 0.0000, 0.0000, 0.0000, 0.0000, 1.5363, 1.9107, 1.4228,
         1.3923, 0.8546, 0.0429, 0.6272, 0.9345, 0.0000, 1.1619],
        [0.0000, 1.5456, 1.4733, 1.8133, 2.5667, 3.1163, 0.8703, 0.2799, 0.7842,